# [Edit Distance](https://leetcode.com/problems/edit-distance/)
## Description
Compute the minimum number of operations required to convert one word to another using insert, delete, or replace.
## Strategy
Use 2D DP table where dp[i][j] = edit distance between first i and j chars.



In [ ]:
export function minDistance(word1: string, word2: string): number {
  return 0;
}

In [ ]:
import { assertEquals } from "https://deno.land/std/testing/asserts.ts";

Deno.test("Edit Distance - Basic", () => {
  assertEquals(minDistance("horse", "ros"), 3);
});

Deno.test("Edit Distance - Same", () => {
  assertEquals(minDistance("abc", "abc"), 0);
});

Deno.test("Edit Distance - Insert All", () => {
  assertEquals(minDistance("", "abc"), 3);
});

